**Outline:**
- Build model and loss function
- Train model
- Observe valdidate
- Test

**To do:**
- Hyperparameter tuning
    + lr
    + layer1, layer2
    + betas

**Modification**
- Weight initialization with xavier uniform
- Adam optimization
- LR decay

In [1]:
import importlib
import argparse
import os, sys
import argparse
import pandas as pd
import numpy as np
import pickle
import time

In [2]:
import torch
import torch.utils.data
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler
from torch.autograd import Variable
from torchvision import transforms

In [3]:
sys.path.insert(0, './../utils/')
from utils import *
from models import *

In [4]:
#Define parser
#name = 'bpi_2012'
name = 'bpi_2013'
#name = 'Road_Traffic_Fine_Management_Process'  

parser = {
    'train': True,
    'test': True,
    'model_class': 'AE',
    'model_name': '',
    'data_dir': '../data/',
    'data_file': name + '.csv',
    'nan_pct': 0.3,
    'input_dir': '../input/{}/'.format(name), 
    'output_dir': './output/{}/'.format(name),
    'batch_size' : 16,
    'epochs' : 100,
    'no_cuda' : False,
    'seed' : 7,
    'layer1': 1000,
    'layer2': 100,
    'lr': 0.002,
    'betas': (0.9, 0.999),   
    'lr_decay': 0.90,
}

args = argparse.Namespace(**parser)

In [5]:
if not os.path.isdir(args.output_dir):
    os.makedirs(args.output_dir)

In [6]:
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [7]:
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [8]:
kwargs = {'num_workers': 2, 'pin_memory': True} if args.cuda else {}

In [9]:
preprocessed_data_name = os.path.join(args.input_dir, 'preprocessed_data_{}.pkl'.format(args.nan_pct))
with open(preprocessed_data_name, 'rb') as f:
    min_max_storage = pickle.load(f)
    complete_matrix_w_normalized_time_train = pickle.load(f)
    missing_matrix_w_normalized_time_train = pickle.load(f)
    avai_matrix_train = pickle.load(f)
    nan_matrix_train = pickle.load(f)
    complete_matrix_w_normalized_time_val = pickle.load(f)
    missing_matrix_w_normalized_time_val = pickle.load(f)
    avai_matrix_val = pickle.load(f)
    nan_matrix_val = pickle.load(f)
    pad_matrix_val = pickle.load(f)
    complete_matrix_w_normalized_time_test = pickle.load(f)
    missing_matrix_w_normalized_time_test = pickle.load(f)
    avai_matrix_test = pickle.load(f)
    nan_matrix_test = pickle.load(f)
    pad_matrix_test = pickle.load(f)
    cols_w_time = pickle.load(f)
    cols_w_normalized_time = pickle.load(f)

In [10]:
file_name = os.path.join(args.input_dir, 'parameters_{}.pkl'.format(args.nan_pct))
with open(file_name, 'rb') as f:
    most_frequent_activity = pickle.load(f)
    first_timestamp = pickle.load(f)
    avai_instance = pickle.load(f)
    nan_instance = pickle.load(f)
    train_size = pickle.load(f)
    val_size = pickle.load(f)
    test_size = pickle.load(f)
    train_row_num = pickle.load(f)
    val_row_num = pickle.load(f)
    test_row_num = pickle.load(f)

# Load data

## Train

In [11]:
complete_matrix_w_normalized_time_trainLoader = torch.utils.data.DataLoader(complete_matrix_w_normalized_time_train, 
                                                                            batch_size=args.batch_size, shuffle=False, 
                                                                            num_workers=2)
missing_matrix_w_normalized_time_trainLoader = torch.utils.data.DataLoader(missing_matrix_w_normalized_time_train, 
                                                                           batch_size=args.batch_size, shuffle=False, 
                                                                           num_workers=2)
avai_matrix_trainLoader = torch.utils.data.DataLoader(avai_matrix_train, 
                                                      batch_size=args.batch_size, shuffle=False, 
                                                      num_workers=2)

## Validate and test

In [12]:
normalized_complete_df_name = os.path.join(args.input_dir, 'normalized_complete_df_{}.csv'.format(args.nan_pct))
normalized_complete_df = pd.read_csv(normalized_complete_df_name)

normalized_missing_df_name = os.path.join(args.input_dir, 'normalized_missing_df_{}.csv'.format(args.nan_pct))
normalized_missing_df = pd.read_csv(normalized_missing_df_name)

In [13]:
missing_true_val = normalized_missing_df[train_row_num:-test_row_num].reset_index(drop=True)
complete_true_val = normalized_complete_df[train_row_num:-test_row_num].reset_index(drop=True)

In [14]:
missing_true_test = normalized_missing_df[-test_row_num:].reset_index(drop=True)
complete_true_test = normalized_complete_df[-test_row_num:].reset_index(drop=True)

In [15]:
missing_true_val.shape, missing_true_test.shape

((1096, 5), (1071, 5))

In [16]:
nan_time_index_val, nan_activity_index_val = getnanindex(missing_true_val)

In [17]:
nan_time_index_test, nan_activity_index_test = getnanindex(missing_true_test)

In [18]:
pd.isnull(normalized_missing_df).sum()

CaseID                  0
Activity             2022
CompleteTimestamp    1974
CumTimeInterval      1974
NormalizedTime       1974
dtype: int64

In [19]:
pd.isnull(missing_true_val).sum()

CaseID                 0
Activity             342
CompleteTimestamp    309
CumTimeInterval      309
NormalizedTime       309
dtype: int64

In [20]:
pd.isnull(missing_true_test).sum()

CaseID                 0
Activity             325
CompleteTimestamp    307
CumTimeInterval      307
NormalizedTime       307
dtype: int64

# Build model

## Define model

In [21]:
if args.model_class == 'AE':
    model = AE(complete_matrix_w_normalized_time_train.shape, args.layer1, args.layer2)

if args.cuda:
    model.cuda()

## Define loss

In [22]:
# Define loss

def loss_function(recon_x, x, avai_mask):
    BCE = F.binary_cross_entropy(recon_x, x, weight=avai_mask, size_average=False) 
    return BCE

In [23]:
optimizer = optim.Adam(model.parameters(), lr=args.lr, betas=args.betas)

In [24]:
#Adjust learning rate per epoch: http://pytorch.org/docs/master/optim.html?highlight=adam#torch.optim.Adam

# Method 1:
#lambda1 = lambda epoch: args.lr_decay ** epoch
#scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=[lambda1])

# Method 2:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10)

## Utils

In [25]:
def save_model(model, epoch, score):
    model_file = os.path.join(args.output_dir, 'model_{}_epoch{}_score{:.4f}.pth'.format(args.model_class, epoch, score))
    torch.save(model.state_dict(), model_file)

In [26]:
def load_model(model, model_name):
    model_file = os.path.join(args.output_dir, model_name)
    assert os.path.isfile(model_file), 'Error: no model found!'
    model_state = torch.load(model_file)
    model.load_state_dict(model_state)

In [27]:
def val(model, missing_matrix_w_normalized_time_val, complete_matrix_w_normalized_time_val, nan_matrix_val):
    model.eval()
    m_val = missing_matrix_w_normalized_time_val
    m_val = Variable(torch.Tensor(m_val).float())
    
    c_val = complete_matrix_w_normalized_time_val
    c_val = Variable(torch.Tensor(c_val).float())
    
    nan_matrix_val = Variable(torch.Tensor(nan_matrix_val).float())
    
    if args.cuda:
        m_val = m_val.cuda()
        c_val = c_val.cuda()
        nan_matrix_val = nan_matrix_val.cuda()
        
    recon_data = model(m_val)
        
    loss = loss_function(recon_data, c_val, nan_matrix_val)
    return loss.data[0]/missing_matrix_w_normalized_time_val.shape[0]

```python
def val(model, missing_matrix_w_normalized_time_val, complete_true_val, missing_true_val,
       pad_matrix_val, cols_w_normalized_time, val_row_num,
       nan_time_index_val, nan_activity_index_val):
    model.eval()
    m_val = missing_matrix_w_normalized_time_val
    m_val = Variable(torch.Tensor(m_val).float())
    
    if args.cuda:
        m_val = m_val.cuda()
        
    recon_val = model(m_val)
    
    recon_df_w_normalized_time = convert2df(recon_val, pad_matrix_val, cols_w_normalized_time, val_row_num)
    recon_df_w_time = getDfWithTime(recon_df_w_normalized_time, missing_true_val, min_max_storage)
    submission_df = getSubmission(recon_df_w_time, missing_true_val, complete_true_val, first_timestamp)
    
    #evaluate
    time, acc = evaluation(submission_df, nan_time_index_val, nan_activity_index_val)
    
    
    return time/86400+1/acc
```

In [28]:
missing_true_val.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime
0,893,NaN,2012-01-14 04:03:40,189490438.0,0.000000
1,893,Accepted-In Progress,2012-01-31 22:16:55,191024833.0,0.507308
2,893,Queued-Awaiting Assignment,NaN,NaN,NaN
3,893,Accepted-In Progress,2012-01-31 23:02:59,191027597.0,0.508222
4,893,Completed-Closed,2012-02-18 04:13:20,192515018.0,1.000000


In [29]:
complete_true_val.head()

,CaseID,Activity,CompleteTimestamp,CumTimeInterval,NormalizedTime
0,893,Queued-Awaiting Assignment,2012-01-14 04:03:40,189490438.0,0.000000
1,893,Accepted-In Progress,2012-01-31 22:16:55,191024833.0,0.507308
2,893,Queued-Awaiting Assignment,2012-01-31 22:29:08,191025566.0,0.507551
3,893,Accepted-In Progress,2012-01-31 23:02:59,191027597.0,0.508222
4,893,Completed-Closed,2012-02-18 04:13:20,192515018.0,1.000000


# Train model

In [30]:
def train(epoch, model, optimizer):
    model.train()
    train_loss = 0
    for batch_idx, (m_data, c_data, avai_mask) in enumerate(zip(missing_matrix_w_normalized_time_trainLoader, 
                                                     complete_matrix_w_normalized_time_trainLoader,
                                                     avai_matrix_trainLoader)):
        c_data = Variable(c_data.float())
        m_data = Variable(m_data.float())
        avai_mask = Variable(avai_mask.float())

        if args.cuda:
            c_data = c_data.cuda()
            m_data = m_data.cuda()
            avai_mask = avai_mask.cuda()

            
        optimizer.zero_grad()
        
        recon_data = model(m_data)
        
        loss = loss_function(recon_data, c_data, avai_mask)
        
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        
    return train_loss / len(complete_matrix_w_normalized_time_trainLoader.dataset)

In [31]:
if args.train:
    for epoch in range(1, args.epochs + 1):
        init = time.time()
        
        #method 1 scheduler
        #scheduler.step()
        train_loss = train(epoch, model, optimizer)
        #val_score = val(model, missing_matrix_w_normalized_time_val, complete_true_val, missing_true_val,
        #                    pad_matrix_val, cols_w_normalized_time, val_row_num, 
        #                    nan_time_index_val, nan_activity_index_val)
        val_score = val(model, missing_matrix_w_normalized_time_val, complete_matrix_w_normalized_time_val, avai_matrix_val)
        
        if epoch == 1:
            current_best = val_score
            save_model(model, epoch, val_score)
        
        else:
            if val_score < current_best:
                current_best = val_score
                save_model(model, epoch, val_score)
        
        #method 2 scheduler
        scheduler.step(val_score)
        
        end = time.time()
        print('====> Epoch {} | Time: {} ms| Train loss: {:.4f} | Val score: {:.4f}'.format(
                          epoch, (end-init)*1000 ,train_loss, val_score))
else:
    load_model(model, args.model_name)

====> Epoch 1 | Time: 1.0503931045532227 | Train loss: 6.3136 | Val score: 2.6610
====> Epoch 2 | Time: 0.9569730758666992 | Train loss: 3.6080 | Val score: 2.0651
====> Epoch 3 | Time: 0.9770889282226562 | Train loss: 2.5382 | Val score: 1.7231
====> Epoch 4 | Time: 0.9992790222167969 | Train loss: 1.9047 | Val score: 1.6004
====> Epoch 5 | Time: 0.9386320114135742 | Train loss: 1.4941 | Val score: 1.5097
====> Epoch 6 | Time: 0.7934360504150391 | Train loss: 1.2358 | Val score: 1.4769
====> Epoch 7 | Time: 0.7972898483276367 | Train loss: 1.0794 | Val score: 1.4542
====> Epoch 8 | Time: 0.7946658134460449 | Train loss: 0.9964 | Val score: 1.5320
====> Epoch 9 | Time: 0.7819509506225586 | Train loss: 0.9117 | Val score: 1.4580
====> Epoch 10 | Time: 0.8105800151824951 | Train loss: 0.8291 | Val score: 1.4232
====> Epoch 11 | Time: 0.8077678680419922 | Train loss: 0.7878 | Val score: 1.4684
====> Epoch 12 | Time: 0.7846889495849609 | Train loss: 0.7568 | Val score: 1.4392
====> Epoch 1

# Predict and evaluate

In [32]:
if args.test:
    m_test = missing_matrix_w_normalized_time_test
    m_test = Variable(torch.Tensor(m_test).float())
    
    if args.cuda:
        m_test = m_test.cuda()
    
    print('Predicting...')
    recon_test = model(m_test)
    
    print('\n')
    print('Converting to dataframe...')
    recon_df_w_normalized_time = convert2df(recon_test, pad_matrix_test, cols_w_normalized_time, test_row_num)
    
    print('Transforming Normalized Time to Time...')
    recon_df_w_time = getDfWithTime(recon_df_w_normalized_time, missing_true_test, min_max_storage)
    
    print('Getting submission...')
    submission_df = getSubmission(recon_df_w_time, missing_true_test, complete_true_test, first_timestamp)
    submission = fixTime(submission_df)
    
    print('Testing...')
    mae_time, rmse_time, acc = evaluation(submission, nan_time_index_test, nan_activity_index_test,show=True)
    print('\n')
    
    print('Saving submission...')
    submission_df.to_csv(args.output_dir+'submission.csv', index=False)
    print('Done!')

Predicting...


Converting to dataframe...
Transforming Normalized Time to Time...
Getting submission...


/Users/hoangnguyen/miniconda3/envs/pydata/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Testing...
Number of missing Time: 307
Mean Absolute Error: 8.0397 day(s)
Root Mean Squared Error: 15.8874 day(s)
Number of missing Activity: 325
Accuracy: 70.46%


Saving submission...
Done!


In [33]:
submission_df.head(10)

,CaseID,TrueActivity,PredictedActivity,TrueTime,PredictedTime,TrueCompleteTimestamp,PredictedCompleteTimestamp
0,1190,Accepted-In Progress,Accepted-In Progress,194135680.0,1.971568e+08,2012-03-07 22:24:22,2012-04-11 21:36:11.001838
1,1190,Completed-Closed,Completed-Closed,197156789.0,1.971568e+08,2012-04-11 21:36:11,2012-04-11 21:36:11
2,1191,Accepted-In Progress,Accepted-In Progress,194135786.0,1.941358e+08,2012-03-07 22:26:08,2012-03-07 22:26:08
3,1191,Completed-Closed,Completed-Closed,198348747.0,1.941358e+08,2012-04-25 16:42:09,2012-03-07 22:26:08.099999
4,1192,Accepted-In Progress,Accepted-In Progress,194141794.0,1.941418e+08,2012-03-08 00:06:16,2012-03-08 00:06:16
5,1192,Completed-Closed,Completed-Closed,194821332.0,1.948213e+08,2012-03-15 20:51:54,2012-03-15 20:51:54
6,1193,Accepted-In Progress,Accepted-In Progress,194163235.0,1.941632e+08,2012-03-08 06:03:37,2012-03-08 06:03:37
7,1193,Accepted-In Progress,Accepted-In Progress,194319293.0,1.943193e+08,2012-03-10 01:24:35,2012-03-10 01:24:35
8,1193,Accepted-Wait,Accepted-Assigned,195953439.0,1.941663e+08,2012-03-28 23:20:21,2012-03-08 06:54:20.199579
9,1193,Accepted-In Progress,Accepted-In Progress,199039446.0,1.941900e+08,2012-05-03 16:33:48,2012-03-08 13:30:16.317510


In [34]:
submission.head(10)

,CaseID,TrueActivity,PredictedActivity,TrueTime,PredictedTime,TrueCompleteTimestamp,PredictedCompleteTimestamp
0,1190,Accepted-In Progress,Accepted-In Progress,194135680.0,1.971568e+08,2012-03-07 22:24:22,2012-04-11 21:36:11.001838
1,1190,Completed-Closed,Completed-Closed,197156789.0,1.971568e+08,2012-04-11 21:36:11,2012-04-11 21:36:11.001838
2,1191,Accepted-In Progress,Accepted-In Progress,194135786.0,1.941358e+08,2012-03-07 22:26:08,2012-03-07 22:26:08
3,1191,Completed-Closed,Completed-Closed,198348747.0,1.941358e+08,2012-04-25 16:42:09,2012-03-07 22:26:08.099999
4,1192,Accepted-In Progress,Accepted-In Progress,194141794.0,1.941418e+08,2012-03-08 00:06:16,2012-03-08 00:06:16
5,1192,Completed-Closed,Completed-Closed,194821332.0,1.948213e+08,2012-03-15 20:51:54,2012-03-15 20:51:54
6,1193,Accepted-In Progress,Accepted-In Progress,194163235.0,1.941632e+08,2012-03-08 06:03:37,2012-03-08 06:03:37
7,1193,Accepted-In Progress,Accepted-In Progress,194319293.0,1.943193e+08,2012-03-10 01:24:35,2012-03-10 01:24:35
8,1193,Accepted-Wait,Accepted-Assigned,195953439.0,1.943193e+08,2012-03-28 23:20:21,2012-03-10 01:24:35
9,1193,Accepted-In Progress,Accepted-In Progress,199039446.0,1.943193e+08,2012-05-03 16:33:48,2012-03-10 01:24:35


In [35]:
#bpi2013
#7.6
#75%

In [36]:
#bpi2012
#0.59
#75%

In [37]:
#helpdesk
#3.9
#85